In [ ]:

from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [ ]:
# torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros')

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 3, padding=0) #input -? OUtput? RF
        self.conv2 = nn.Conv2d(16,16, 3, padding=0)
        
        self.BN = nn.BatchNorm2d(num_features=16, eps=1e-05, momentum=0.1)
        self.BN1 = nn.BatchNorm2d(num_features=32, eps=1e-05, momentum=0.1)
        self.drop  = nn.Dropout(0.1)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(16, 16, 1, padding=0)
        # self.drop  = nn.Dropout(0.1)
        self.conv4 = nn.Conv2d(16, 32, 3, padding=0)
        self.drop  = nn.Dropout(0.1)
        
        # self.pool2 = nn.MaxPool2d(2, 2)
        self.conv5 = nn.Conv2d(32, 16, 3)
        # self.drop  = nn.Dropout(0.1)
        self.conv6 = nn.Conv2d(16, 10, 3)
        # self.conv7 = nn.Conv2d(10, 10, 1)

    def forward(self, x):
        x = self.pool1(self.drop(self.BN(F.relu(self.conv2(F.relu(self.conv1(x)))))))
        x = self.pool1(self.drop(F.relu(self.BN1(self.conv4(self.drop(F.relu(self.conv3(x))))))))
        x = self.conv6(self.drop(F.relu(self.conv5(x))))
        # x = self.conv7(x)
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [ ]:
# !pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 26, 26]             160
            Conv2d-2           [-1, 16, 24, 24]           2,320
       BatchNorm2d-3           [-1, 16, 24, 24]              32
           Dropout-4           [-1, 16, 24, 24]               0
         MaxPool2d-5           [-1, 16, 12, 12]               0
            Conv2d-6           [-1, 16, 12, 12]             272
           Dropout-7           [-1, 16, 12, 12]               0
            Conv2d-8           [-1, 32, 10, 10]           4,640
       BatchNorm2d-9           [-1, 32, 10, 10]              64
          Dropout-10           [-1, 32, 10, 10]               0
        MaxPool2d-11             [-1, 32, 5, 5]               0
           Conv2d-12             [-1, 16, 3, 3]           4,624
          Dropout-13             [-1, 16, 3, 3]               0
           Conv2d-14             [-1, 1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:29: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [ ]:

""

torch.manual_seed(1)
batch_size = 64
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)

In [ ]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.4f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * float(correct) / float(len(test_loader.dataset))))

In [ ]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 15):
    print(epoch)
    train(model, device, train_loader, optimizer, 512)
    test(model, device, test_loader)


  0%|          | 0/938 [00:00<?, ?it/s]

1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:29: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  0%|          | 0/938 [00:00<?, ?it/s]


Test set: Average loss: 0.0530, Accuracy: 9838/10000 (98.3800%)

2


  0%|          | 0/938 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0262, Accuracy: 9910/10000 (99.1000%)

13


  0%|          | 0/938 [00:00<?, ?it/s]


Test set: Average loss: 0.0237, Accuracy: 9923/10000 (99.2300%)

14


loss=0.0057672858238220215 batch_id=937: 100%|██████████| 938/938 [00:16<00:00, 62.21it/s]



Test set: Average loss: 0.0242, Accuracy: 9917/10000 (99.1700%)



In [ ]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 20):
    print(epoch)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)